In [102]:
import pandas as pd
from sklearn import preprocessing
import torch
from torch import optim

from utils import utils
DEVICE = utils.DEVICE

# 0. Prep Data

In [103]:
source_idx = 2
target_idx = 3
winter_idx = 0
summer_idx = 0

train_source_X = pd.read_csv(f"./deep_occupancy_detection/data/{source_idx}_X_train.csv")
target_X = pd.read_csv(f"./deep_occupancy_detection/data/{target_idx}_X_train.csv")
train_source_y_task = pd.read_csv(f"./deep_occupancy_detection/data/{source_idx}_Y_train.csv")[train_source_X.Season==winter_idx].values.reshape(-1)
target_y_task = pd.read_csv(f"./deep_occupancy_detection/data/{target_idx}_Y_train.csv")[target_X.Season==summer_idx].values.reshape(-1)
train_source_X = train_source_X[train_source_X.Season==winter_idx]
target_X = target_X[target_X.Season==summer_idx]

In [104]:
train_source_X

,Energy,Season,Time
2176,168649.947,0,12
2177,169192.356,0,13
2178,168560.052,0,14
2179,130375.163,0,15
2180,132788.967,0,16
...,...,...,...
3131,996765.945,0,39
3132,1305499.967,0,40
3133,886801.848,0,41
3134,991335.091,0,42


In [105]:
target_X

,Energy,Season,Time
0,380827.893,1,12
1,194841.812,1,13
2,122493.428,1,14
3,155853.417,1,15
4,158295.543,1,16
...,...,...,...
2171,200783.630,1,39
2172,197410.112,1,40
2173,195676.564,1,41
2174,195884.090,1,42


In [106]:
scaler = preprocessing.StandardScaler()
train_source_X = scaler.fit_transform(train_source_X)
target_X = scaler.fit_transform(target_X)

train_source_X, train_source_y_task = utils.apply_sliding_window(train_source_X, train_source_y_task, filter_len=6)
target_X, target_y_task = utils.apply_sliding_window(target_X, target_y_task, filter_len=6)

In [107]:
source_loader, _, _, _, target_X, target_y_task = utils.get_loader(train_source_X, target_X, train_source_y_task, target_y_task, shuffle=True)

In [108]:
test_thr = int(target_X.shape[0]/2)
test_target_X = target_X[test_thr:]
test_target_y_task = target_y_task[test_thr:]

# 1. p(Y|X) Divergence
## 1.1 Model Fit


In [109]:
feature_extractor = utils.Conv1d(input_size=train_source_X.shape[2]).to(DEVICE)
task_classifier = utils.Decoder(input_size=128, output_size=1).to(DEVICE)
criterion = torch.nn.BCELoss()

feature_optimizer = optim.Adam(feature_extractor.parameters(), lr=0.0001)
task_optimizer = optim.Adam(task_classifier.parameters(), lr=0.0001)

In [110]:
for epoch in range(300):
    for source_data in source_loader:
        source_X, source_y = source_data
        source_y_task = source_y[:, 0]

        source_feature = feature_extractor(source_X)
        source_output = task_classifier(source_feature)
        source_output = torch.sigmoid(source_output).reshape(-1)

        loss = criterion(source_output, source_y_task)

        feature_optimizer.zero_grad()
        task_optimizer.zero_grad()
        loss.backward()
        feature_optimizer.step()
        task_optimizer.step()
    if epoch % 50 == 0:
        print(f"Loss: {loss.item()}")

Loss: 0.6239587068557739


Loss: 0.17412421107292175
Loss: 1.832610011100769
Loss: 0.6329265236854553
Loss: 1.1220811605453491
Loss: 0.7572677135467529


## 1.2 Calc Accuracy for Target

In [111]:
pred_y = task_classifier(feature_extractor(target_X))
pred_y = torch.sigmoid(pred_y).reshape(-1)
pred_y = pred_y > 0.5
acc = sum(pred_y == target_y_task) / pred_y.shape[0]
print(f"Accuracy for Target: {acc}")

Accuracy for Target: 0.8843850493431091
